# Metode Klasifikasi Algoritma Naive Bayes

Anggota Kelompok 4 :
- Muhammad Richo Abadinata     (215150200111037)
- Ade Arya Dwi Syahputra       (215150207111002)
- Ikrar Budidharma 				     (215150207111011)

## Dataset

Import Library

In [75]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.datasets import make_classification

from sklearn.metrics import classification_report

Import Dataset

In [76]:
from google.colab import files
heart = files.upload()

Saving heart.csv to heart (1).csv


In [77]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('heart.csv')
dataset

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


Mengubah atribut data yang berupa kategorical menjadi numerical

In [78]:
dataset['Sex'].replace(['M', 'F'],[0, 1], inplace=True)
dataset['ChestPainType'].replace(['TA', 'ATA', 'NAP', 'ASY'],[0, 1, 2, 3], inplace=True)
dataset['RestingECG'].replace(['Normal', 'ST', 'LVH'],[0, 1, 2], inplace=True)
dataset['ExerciseAngina'].replace(['Y','N'],[0, 1], inplace=True)
dataset['ST_Slope'].replace(['Up','Flat','Down'],[0, 1, 2], inplace=True)

Mengecek apakah sudah atribut dataset sudah terubah 

In [79]:
dataset.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,1,140,289,0,0,172,1,0.0,0,0
1,49,1,2,160,180,0,0,156,1,1.0,1,1
2,37,0,1,130,283,0,1,98,1,0.0,0,0
3,48,1,3,138,214,0,0,108,0,1.5,1,1
4,54,0,2,150,195,0,0,122,1,0.0,0,0


In [80]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    int64  
 2   ChestPainType   918 non-null    int64  
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    int64  
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    int64  
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    int64  
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(11)
memory usage: 86.2 KB


Data Imputasi

In [81]:
def imputasi(df_input):
    column = df_input.select_dtypes(float).columns
    class_column = df_input.columns[-1]
    for item in column:
        df_input[item].fillna(df_input.groupby(class_column)[item].transform('mean').round(2), inplace=True)
    return df_input

In [82]:
imputasi(dataset).isnull().values.any()

False

In [83]:
def cetak_rentang(df_input):
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    print("Rentang fitur",fitur,"adalah",max-min)

In [84]:
cetak_rentang(dataset)

Rentang fitur Age adalah 49
Rentang fitur Sex adalah 1
Rentang fitur ChestPainType adalah 3
Rentang fitur RestingBP adalah 200
Rentang fitur Cholesterol adalah 603
Rentang fitur FastingBS adalah 1
Rentang fitur RestingECG adalah 2
Rentang fitur MaxHR adalah 142
Rentang fitur ExerciseAngina adalah 1
Rentang fitur Oldpeak adalah 8.8
Rentang fitur ST_Slope adalah 2


Data Z-Score

In [85]:
def zscore(df_input):
  list_fitur = df_input.columns[:-1]
  for item in list_fitur:
    item_mean = df_input[item].mean()
    item_std = df_input[item].std()
    df_input[item] = (df_input[item] - item_mean) / item_std
  return df_input

In [86]:
heart_zscore = zscore(dataset)

In [87]:
heart_zscore

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,-1.432359,-0.515671,-1.344353,0.410685,0.824621,-0.551041,-0.748772,1.382175,0.823108,-0.831979,-1.051541,0
1,-0.478223,1.937107,-0.270275,1.490940,-0.171867,-0.551041,-0.748772,0.753746,0.823108,0.105606,0.595753,1
2,-1.750404,-0.515671,-1.344353,-0.129442,0.769768,-0.551041,0.491973,-1.524307,0.823108,-0.831979,-1.051541,0
3,-0.584238,1.937107,0.803804,0.302660,0.138964,-0.551041,-0.748772,-1.131539,-1.213585,0.574398,0.595753,1
4,0.051853,-0.515671,-0.270275,0.950812,-0.034736,-0.551041,-0.748772,-0.581664,0.823108,-0.831979,-1.051541,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,-0.902283,-0.515671,-2.418431,-1.209697,0.596068,-0.551041,-0.748772,-0.188897,0.823108,0.293123,0.595753,1
914,1.536064,-0.515671,0.803804,0.626736,-0.053020,1.812770,-0.748772,0.164595,0.823108,2.355810,0.595753,1
915,0.369898,-0.515671,0.803804,-0.129442,-0.619830,-0.551041,-0.748772,-0.856602,-1.213585,0.293123,0.595753,1
916,0.369898,1.937107,-1.344353,-0.129442,0.340090,-0.551041,1.732717,1.460728,0.823108,-0.831979,0.595753,1


In [88]:
heart_zscore.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,918.000000,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,918.000000
mean,-1.393221e-16,6.966105e-17,-6.192094e-17,2.031781e-16,3.096047e-17,4.644070e-17,0.000000,5.263280e-16,3.483053e-17,1.083616e-16,-3.096047e-17,0.553377
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.497414
min,-2.704540e+00,-5.156713e-01,-2.418431e+00,-7.151097e+00,-1.817444e+00,-5.510410e-01,-0.748772,-3.016825e+00,-1.213585e+00,-3.269699e+00,-1.051541e+00,0.000000
25%,-6.902531e-01,-5.156713e-01,-2.702746e-01,-6.695696e-01,-2.335765e-01,-5.510410e-01,-0.748772,-6.602179e-01,-1.213585e+00,-8.319789e-01,-1.051541e+00,0.000000
50%,5.185271e-02,-5.156713e-01,8.038037e-01,-1.294423e-01,2.212426e-01,-5.510410e-01,-0.748772,4.676419e-02,8.231076e-01,-2.694280e-01,5.957534e-01,1.000000
75%,6.879435e-01,-5.156713e-01,8.038037e-01,4.106850e-01,6.234947e-01,-5.510410e-01,0.491973,7.537463e-01,8.231076e-01,5.743984e-01,5.957534e-01,1.000000
max,2.490201e+00,1.937107e+00,8.038037e-01,3.651449e+00,3.695238e+00,1.812770e+00,1.732717,2.560478e+00,8.231076e-01,4.981047e+00,2.243047e+00,1.000000


Train Test Split 


In [89]:
from sklearn.model_selection import train_test_split
array = dataset.values
X = array[:,0:11]
Y = array[:,11]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.2, random_state=5, shuffle=True)

## KNN

Prediksi KNN

In [90]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

In [91]:
##Parameter 
heart_KNN = KNeighborsClassifier(algorithm = 'auto', leaf_size = 30, n_neighbors = 11, p = 1, weights = 'distance')
#K-Fold Sebanyak 5 Kali
for  train_index, test_index in  skf.split(X, Y):
    X_train, X_validation = X[train_index], X[test_index]
    Y_train, Y_validation = Y[train_index], Y[test_index]
    
    heart_KNN.fit(X_train, Y_train)

    predict_KNN = heart_KNN.predict(X_validation)

    print(classification_report(Y_validation, predict_KNN))


              precision    recall  f1-score   support

         0.0       0.87      0.80      0.84        82
         1.0       0.85      0.90      0.88       102

    accuracy                           0.86       184
   macro avg       0.86      0.85      0.86       184
weighted avg       0.86      0.86      0.86       184

              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87        82
         1.0       0.89      0.89      0.89       102

    accuracy                           0.88       184
   macro avg       0.88      0.88      0.88       184
weighted avg       0.88      0.88      0.88       184

              precision    recall  f1-score   support

         0.0       0.83      0.82      0.82        82
         1.0       0.85      0.86      0.86       102

    accuracy                           0.84       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.84      0.84      0.84       184

              preci

In [92]:
##Parameter 
heart_KNN = KNeighborsClassifier(algorithm = 'ball_tree', leaf_size = 30, n_neighbors = 11, p = 1, weights = 'uniform')
#K-Fold Sebanyak 5 Kali
for  train_index, test_index in  skf.split(X, Y):
    X_train, X_validation = X[train_index], X[test_index]
    Y_train, Y_validation = Y[train_index], Y[test_index]
    
    heart_KNN.fit(X_train, Y_train)

    predict_KNN = heart_KNN.predict(X_validation)

    print(classification_report(Y_validation, predict_KNN))

              precision    recall  f1-score   support

         0.0       0.88      0.80      0.84        82
         1.0       0.85      0.91      0.88       102

    accuracy                           0.86       184
   macro avg       0.87      0.86      0.86       184
weighted avg       0.87      0.86      0.86       184

              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87        82
         1.0       0.89      0.89      0.89       102

    accuracy                           0.88       184
   macro avg       0.88      0.88      0.88       184
weighted avg       0.88      0.88      0.88       184

              precision    recall  f1-score   support

         0.0       0.82      0.83      0.82        82
         1.0       0.86      0.85      0.86       102

    accuracy                           0.84       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.84      0.84      0.84       184

              preci

In [93]:
##Parameter 
heart_KNN = KNeighborsClassifier(algorithm = 'kd_tree', leaf_size = 30, n_neighbors = 11, p = 1,  weights = 'uniform')
#K-Fold Sebanyak 5 Kali
for  train_index, test_index in  skf.split(X, Y):
    X_train, X_validation = X[train_index], X[test_index]
    Y_train, Y_validation = Y[train_index], Y[test_index]
    
    heart_KNN.fit(X_train, Y_train)

    predict_KNN = heart_KNN.predict(X_validation)

    print(classification_report(Y_validation, predict_KNN))

              precision    recall  f1-score   support

         0.0       0.88      0.80      0.84        82
         1.0       0.85      0.91      0.88       102

    accuracy                           0.86       184
   macro avg       0.87      0.86      0.86       184
weighted avg       0.87      0.86      0.86       184

              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87        82
         1.0       0.89      0.89      0.89       102

    accuracy                           0.88       184
   macro avg       0.88      0.88      0.88       184
weighted avg       0.88      0.88      0.88       184

              precision    recall  f1-score   support

         0.0       0.82      0.83      0.82        82
         1.0       0.86      0.85      0.86       102

    accuracy                           0.84       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.84      0.84      0.84       184

              preci

In [94]:
##Parameter 
heart_KNN = KNeighborsClassifier(algorithm = 'brute', leaf_size = 30, n_neighbors = 11, p = 1,  weights = 'uniform')
#K-Fold Sebanyak 5 Kali
for  train_index, test_index in  skf.split(X, Y):
    X_train, X_validation = X[train_index], X[test_index]
    Y_train, Y_validation = Y[train_index], Y[test_index]
    
    heart_KNN.fit(X_train, Y_train)

    predict_KNN = heart_KNN.predict(X_validation)

    print(classification_report(Y_validation, predict_KNN))

              precision    recall  f1-score   support

         0.0       0.88      0.80      0.84        82
         1.0       0.85      0.91      0.88       102

    accuracy                           0.86       184
   macro avg       0.87      0.86      0.86       184
weighted avg       0.87      0.86      0.86       184

              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87        82
         1.0       0.89      0.89      0.89       102

    accuracy                           0.88       184
   macro avg       0.88      0.88      0.88       184
weighted avg       0.88      0.88      0.88       184

              precision    recall  f1-score   support

         0.0       0.82      0.83      0.82        82
         1.0       0.86      0.85      0.86       102

    accuracy                           0.84       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.84      0.84      0.84       184

              preci